# PhotometryLib

## Output directories

In [1]:
import os
outdir = os.path.join(os.environ["HOME"], "notebook_output")
csvdir = os.path.join(outdir, "csv")
pngdir = os.path.join(outdir, "png")
! mkdir -p $csvdir                          
! mkdir -p $pngdir                          
! ls $outdir

csv  png


## Importing dependencies

In [2]:
from pathlib import Path
from astropy.io import fits
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)
from RTAscience.cfg.Config import Config
from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.lib.datasource.photometry import Photometry
from rtapipe.lib.plotting.photometryplot import PhotometrySinglePlot, PhotometrySubPlots


Bad key "text.kerning_factor" on line 4 in
/data01/home/baroncelli/.conda/envs/bphd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


## Utility function

In [3]:
def getInput(dataDir, index):
    if index == 0:
        index = 1
    simFolder = datapath.joinpath("obs",dataDir)
    cfg = Config(simFolder.joinpath("config.yaml"))
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    if cfg.get("simtype") == 'bkg':
        inputFitsFile = str(simFolder.joinpath("backgrounds",f"bkg{str(index).zfill(6)}.fits"))
    else:
        inputFitsFile = str(simFolder.joinpath(runid,f"ebl{str(index).zfill(6)}.fits"))
    
    
    conf = {
        'input_file': inputFitsFile,
        'output_dir': outdir,
        'simtype' : cfg.get('simtype'),
        'runid' : cfg.get('runid'),
        't_window_start': 0,
        't_window_stop': cfg.get('tobs'),
        'e_window_start': cfg.get('emin'),
        'e_window_stop': cfg.get('emax'),
        'onset' : cfg.get('onset')
    }
    
    print(f"File: {inputFitsFile}, pointing: {pointing}")

    return inputFitsFile, conf, pointing

## Data folders (simulations)
    folder/
        config.yaml
        backgrounds/
            bkg000001.fits         => simulation background only
            ..
            bkg00000<N>.fits
        <runId>/
            ebl000001.fits         => simulation background + source
            ..
            ebl000001<N>.fits
            

## How to get data

### Data directories
Simulations from template: run0406_ID000126

In [4]:
bkgsrc      = "simtype_grb_os_1800_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5"
bkgonly     = "simtype_bkg_os_0_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5"

### Get csv input, simulation parameters and pointing (from template)

In [5]:
inputFile_srcbkg_0, sim_params_bkgsrc, pointing_srcbkg_0 = getInput(bkgsrc, 1)

File: /data01/home/baroncelli/phd/DATA/obs/simtype_grb_os_1800_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits, pointing: (33.057, -51.841)


In [6]:
inputFile_onlybkg_0, sim_params_bkgonly, pointing_onlybkg_0 = getInput(bkgonly, 1)

File: /data01/home/baroncelli/phd/DATA/obs/simtype_bkg_os_0_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits, pointing: (33.057, -51.841)


### Differences between configuration dictionaries

In [7]:
assert pointing_srcbkg_0 == pointing_onlybkg_0
assert inputFile_srcbkg_0 != inputFile_onlybkg_0
for key, val in sim_params_bkgsrc.items():
    if val!=sim_params_bkgonly[key]:
        print(f"{key}:\n\t{val}\n\t{sim_params_bkgonly[key]}")

input_file:
	/data01/home/baroncelli/phd/DATA/obs/simtype_grb_os_1800_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
	/data01/home/baroncelli/phd/DATA/obs/simtype_bkg_os_0_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
simtype:
	grb
	bkg
onset:
	1800
	0


## The photometry class

In [8]:
photometry = Photometry()

## Photometry parameters

In [9]:
photometry_params = {
    
    # integration parameters for time
    't_window_size': 25,
    't_window_step': 25,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : pointing_srcbkg_0,
    'region_radius': 0.5,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

## Generating some data

### Integrate on time 

In [10]:
csv_data_t_bkg_src = photometry.integrate(photometry_params, sim_params_bkgsrc, integration="t").pop(0)
#csv_data_t_bkg_src = "/data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits"
pd.read_csv(csv_data_t_bkg_src).head(3)

100%|██████████| 144/144 [00:01<00:00, 74.32it/s]


Total counts = 27613, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_t.csv


,VALMIN,VALMAX,VALCENTER,COUNTS,ERROR
0,0,25,12.5,168,12.9615
1,25,50,37.5,177,13.3041
2,50,75,62.5,165,12.8452


In [11]:
csv_data_t_bkg_only = photometry.integrate(photometry_params, sim_params_bkgonly, integration="t").pop(0)
# csv_data_t_bkg_only = "/data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits"
print(pd.read_csv(csv_data_t_bkg_only))

100%|██████████| 144/144 [00:01<00:00, 99.15it/s] 

Total counts = 25287, Produced: /data01/home/baroncelli/notebook_output/csv/bkg000001_simtype_bkg_t.csv
     VALMIN  VALMAX  VALCENTER  COUNTS    ERROR
0         0      25       12.5     180  13.4164
1        25      50       37.5     168  12.9615
2        50      75       62.5     171  13.0767
3        75     100       87.5     176  13.2665
4       100     125      112.5     189  13.7477
..      ...     ...        ...     ...      ...
139    3475    3500     3487.5     165  12.8452
140    3500    3525     3512.5     187  13.6748
141    3525    3550     3537.5     184  13.5647
142    3550    3575     3562.5     184  13.5647
143    3575    3600     3587.5     161  12.6886

[144 rows x 5 columns]


### Integrate on energy

In [12]:
csv_data_e_bkg_src = photometry.integrate(photometry_params, sim_params_bkgsrc, integration="e").pop(0)

100%|██████████| 119/119 [00:01<00:00, 81.02it/s]

Total counts = 27580, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_e.csv


In [13]:
csv_data_e_bkg_only = photometry.integrate(photometry_params, sim_params_bkgonly, integration="e").pop(0)

100%|██████████| 119/119 [00:01<00:00, 94.47it/s]

Total counts = 25262, Produced: /data01/home/baroncelli/notebook_output/csv/bkg000001_simtype_bkg_e.csv


## Plotting the data

### Time integration

In [14]:
singleplot_t = PhotometrySinglePlot(title = "Time integration")
_ = singleplot_t.addData(csv_data_t_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "t")
_ = singleplot_t.addData(csv_data_t_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
multiplot_t = PhotometrySubPlots(title = "Time integration")
_ = multiplot_t.addData(csv_data_t_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "t")
_ = multiplot_t.addData(csv_data_t_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "t")

## Energy integration

In [ ]:
singleplot_e = PhotometrySinglePlot(title = "Energy integration")
singleplot_e.addData(csv_data_e_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "e")
singleplot_e.addData(csv_data_e_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "e")

In [ ]:
multiplot_e = PhotometrySubPlots(title = "Energy integration")
_ = multiplot_e.addData(csv_data_e_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "e")
_ = multiplot_e.addData(csv_data_e_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "e")

## Integration on time with multiple window sizes and background baseline

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple window sizes")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple window sizes")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title="Integration on time with multiple window sizes and background baseline")

photometry_params_tmp = photometry_params.copy()

for ws in [10, 25, 50, 75]:

    # source+background
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["t_window_size","t_window_step"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["baseline (only background)"], "t", as_baseline=True)
    

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_ws_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on energy with multiple window sizes and background baseline

In [15]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title = "Integration on energy with multiple window sizes")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title = "Integration on energy with multiple window sizes")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title = "Integration on energy with multiple window sizes and background baseline")

photometry_params_tmp = photometry_params.copy()

for ee in [0.001, 0.01, 0.02, 0.03]:

    # source+background
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="e").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="e").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["e_window_size","e_window_step"], "e")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["e_window_size","e_window_step"], "e")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "e")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["t_window_size","t_window_step"], "e", as_baseline=True)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  4%|▍         | 5/119 [00:00<00:02, 49.55it/s]

Total counts = 27580, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_e.csv


...

Total counts = 27613, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_e.csv
Total counts = 25287, Produced: /data01/home/baroncelli/notebook_output/csv/bkg000001_simtype_bkg_e.csv


  0%|          | 0/4 [00:00<?, ?it/s]

Total counts = 27613, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_e.csv
Total counts = 25287, Produced: /data01/home/baroncelli/notebook_output/csv/bkg000001_simtype_bkg_e.csv


100%|██████████| 4/4 [00:00<00:00, 28.89it/s]


Total counts = 27613, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_e.csv
Total counts = 25287, Produced: /data01/home/baroncelli/notebook_output/csv/bkg000001_simtype_bkg_e.csv


In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_e_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_e_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_e_ws_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on time with multiple region radius values and background baseline

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple region radius values")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple region radius values")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title="Integration on time with multiple region radius values and background baseline")

photometry_params_tmp = photometry_params.copy()

for rr in [0.1, 1.0, 2.0, 3.0]:

    # source+background
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["region_radius"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["region_radius"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["region_radius"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["region_radius"], "t", as_baseline=True)
    

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_rr_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_rr_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_rr_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on time with multiple pointing values and baseline on original pointing

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple pointing values")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple pointing values")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc_1 = PhotometrySubPlots(title="Integration on time with multiple pointing values and baseline on background")

# fouth plot: background + source on multiple window sizes (separate subplots with original pointing baseline)
multiplot_bgksrc_2 = PhotometrySubPlots(title="Integration on time with multiple pointing values and baseline on original pointing")


photometry_params_tmp = photometry_params.copy()
pointing=pointing_srcbkg_0
pointing_offset = 0.5
pointings = [
    (pointing_srcbkg_0[0]+pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]+pointing_offset), 
    (pointing[0]-pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]-pointing_offset)
] 

photometry_params_tmp2 = photometry_params.copy()
csv_data_t_orig_point = photometry.integrate(photometry_params_tmp2, sim_params_bkgsrc, integration="t").pop(0)

for pp in pointings:

    # source+background
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")

    # background only
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["pointing"], "t")
    
    # source+background + baseline background only
    _ = multiplot_bgksrc_1.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")
    _ = multiplot_bgksrc_1.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, [f"baseline (background only)"], "t", as_baseline=True)    
    
    """
        @FIXME !!!
    """
    # source+background + baseline original pointing
    _ = multiplot_bgksrc_2.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")
    _ = multiplot_bgksrc_2.addData(csv_data_t_orig_point, photometry_params_tmp2, sim_params_bkgsrc, [f"baseline {pointing}"], "t", as_baseline=True)

print(f"Original pointing: {pointing_srcbkg_0}")


In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_p_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_p_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_p_bgksrc"))
multiplot_bgksrc.destroy()

## Plotting different trials

### bkg + src

In [ ]:
bkg_src_files = []
for i in range(4):
    bkg_src_files.append(getInput(bkgsrc, i))

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
multiplot = PhotometrySubPlots(title="Multiple trials")

for i in range(4):
    conf = bkg_src_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)
    _ = singleplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    _ = multiplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    

### Bkg only

In [ ]:
bkg_only_files = []
for i in range(4):
    bkg_only_files.append(getInput(bkgonly, i))

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
multiplot = PhotometrySubPlots(title="Multiple trials")

for i in range(4):
    conf = bkg_only_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)
    _ = singleplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    _ = multiplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")

## Integrate on time and on energy

In [16]:
sim_params_bkgsrc_tmp = sim_params_bkgsrc.copy()
sim_params_bkgsrc_tmp["t_window_start"] = 1000 

photometry_params_tmp = photometry_params.copy()
photometry_params_tmp["t_window_size"] = 50
photometry_params_tmp["t_window_step"] = 50
photometry_params_tmp["e_window_size"] = 0.01
photometry_params_tmp["e_window_step"] = 0.01

In [35]:
csv_files = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc_tmp, integration="et")

0it [00:00, ?it/s]
100%|██████████| 52/52 [00:00<00:00, 58.54it/s]
1it [00:00,  1.12it/s]
 13%|█▎        | 7/52 [00:00<00:00, 62.36it/s]

Total counts = 5933, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.03_0.04.csv



100%|██████████| 52/52 [00:00<00:00, 73.55it/s]
2it [00:01,  1.27it/s]
 17%|█▋        | 9/52 [00:00<00:00, 82.97it/s]

Total counts = 5006, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.04_0.05.csv



100%|██████████| 52/52 [00:00<00:00, 83.53it/s]
3it [00:02,  1.40it/s]
 17%|█▋        | 9/52 [00:00<00:00, 88.22it/s]

...

Total counts = 382, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.12_0.13.csv



100%|██████████| 52/52 [00:00<00:00, 87.34it/s]
11it [00:07,  1.51it/s]
 17%|█▋        | 9/52 [00:00<00:00, 89.63it/s]

Total counts = 310, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.13_0.14.csv



100%|██████████| 52/52 [00:00<00:00, 86.21it/s]
12it [00:08,  1.49it/s]

Total counts = 285, Produced: /data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.14_0.15.csv


In [36]:
csv_files_to_plot = [csv_files[0], csv_files[3], csv_files[-4], csv_files[-1]]
csv_files_to_plot

['/data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.03_0.04.csv',
 '/data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.06_0.07.csv',
 '/data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.11_0.12.csv',
 '/data01/home/baroncelli/notebook_output/csv/ebl000001_simtype_grb_f_energy_window_0.14_0.15.csv']

In [30]:
def exctractInterval(f):
    emin,emax = f.split("energy_window_")[1].split(".csv")[0].split("_")
    return emin,emax

In [37]:
singleplot_t = PhotometrySinglePlot(title = "Energy/Time integration")
for f in csv_files_to_plot:
    emin, emax = exctractInterval(f)
    _ = singleplot_t.addData(f, photometry_params_tmp, sim_params_bkgsrc_tmp, [f"emin: {emin} emax: {emax}"], "t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# TODO

* fix simulation outdir name (it should include other params) (done)
* onset moved into template params (done)
* Geneare background più corti (done)
* custom titles (done)
* refactoring: own classes, getTitle() and getLabel() in base class (done)
* plottare differenti trials (done)
* refactoring photometry (done)
* round a 4 decimali (done)
* et, te integrations (done)
* plottare et (done)

* bug pointing as baseline
* plottare differenti runid